In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, ExtraTreesClassifier
# from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb

In [2]:
from IPython.display import Audio, display

def allDone():
    display(Audio(url='https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav', autoplay=True))

In [3]:
train = pd.read_csv('data/train.csv', index_col='Unnamed: 0', sep='\t')
test = pd.read_csv('data/test.csv', index_col='Unnamed: 0', sep='\t')

xtrain = train.drop('0', axis=1)
ytrain = train['0']
xtest = test.drop('0', axis=1)
ytest = test['0']

In [4]:
def pred_to_file(pred):
    pred = pd.DataFrame(pred, columns=["_VAL_"])
    pred.to_csv('solution_3.csv', index_label="_ID_")

In [5]:
cols_to_drop = ['9', '140', '164', '11', '5', '129', '130', '137', '138', '141', '149', '150', '178', '186', '188',
                '192', '193', '291', '301', '303', '305', '152', '160', '191', '182', '185', '181', '172', '170',
                '157', '136', '135']

xtrain.drop(cols_to_drop, axis=1, inplace=True)
xtest.drop(cols_to_drop, axis=1, inplace=True)

In [6]:
X_train, X_holdout, y_train, y_holdout = train_test_split(xtrain, ytrain,
                                                    test_size=0.20,
                                                    random_state=42, stratify=ytrain)

In [7]:
kf = KFold(n_splits=10)
skf = StratifiedKFold(n_splits=10)

In [12]:
estimator = LogisticRegression(random_state=666)

params = {
    'C' : (0.1, 0.15, 0.2, 0.25, 0.5, 0.8, 1)
}

gs = GridSearchCV(
    estimator=estimator,
    param_grid=params,
    cv=skf,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

gs.fit(X=X_train,y=y_train)

best_lr = gs.best_estimator_
best_score = gs.best_score_

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   32.9s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:  1.0min finished


In [23]:
print("Baseline = ", best_score)

Baseline =  0.7275376894594372


In [88]:
pred_to_file(best_lr.predict_proba(xtest)[:, 1])

### xgboost

In [8]:
params = {
    #default
    'objective': 'reg:logistic',
    'eta': 0.1,
    'silent': 1,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": 'auc'
}

xgb_train = xgb.DMatrix(xtrain, ytrain, feature_names=xtrain.columns)

In [10]:
results = xgb.cv(params,xgb_train,
                 num_boost_round=200,
                 early_stopping_rounds=10,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.708571+0.00546898	test-auc:0.679583+0.0062847
[1]	train-auc:0.728341+0.00466282	test-auc:0.695084+0.0080493
[2]	train-auc:0.737702+0.00199645	test-auc:0.701467+0.00764415
[3]	train-auc:0.742555+0.0025073	test-auc:0.704987+0.00635292
[4]	train-auc:0.747133+0.00321745	test-auc:0.707569+0.00700152
[5]	train-auc:0.751657+0.00380819	test-auc:0.7102+0.00850609
[6]	train-auc:0.755208+0.00197601	test-auc:0.711661+0.00596181
[7]	train-auc:0.759184+0.00272927	test-auc:0.712976+0.00621004
[8]	train-auc:0.762415+0.00243296	test-auc:0.713888+0.0053963
[9]	train-auc:0.765591+0.00288591	test-auc:0.714246+0.00527878
[10]	train-auc:0.768442+0.00292221	test-auc:0.714146+0.00480181
[11]	train-auc:0.771464+0.00327101	test-auc:0.715015+0.00491449
[12]	train-auc:0.77469+0.00307474	test-auc:0.716108+0.00455814
[13]	train-auc:0.777802+0.00327215	test-auc:0.716749+0.00418458
[14]	train-auc:0.780486+0.00329337	test-auc:0.717924+0.00418925
[15]	train-auc:0.784232+0.00307153	test-auc:0.718966+0.00

In [61]:
params = {
    #default
    'objective': 'reg:logistic',
    'eta': 0.05,
    'silent': 1,
    "nthread": -1,
    "random_seed": 666,
    "eval_metric": 'auc',

    # regularization parameters
#     'max_leavs': 16,
#     'max_depth': 6,
    'gamma': 8,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    
    #lightgbm approach
    'tree_method': 'hist'
}

results = xgb.cv(params,xgb_train,
                 num_boost_round=1000,
                 early_stopping_rounds=18,
                 folds=skf, verbose_eval=1)

[0]	train-auc:0.69543+0.00412001	test-auc:0.674409+0.0081172
[1]	train-auc:0.714987+0.00394338	test-auc:0.690508+0.00213083
[2]	train-auc:0.722343+0.00444996	test-auc:0.695188+0.00585077
[3]	train-auc:0.726942+0.00142743	test-auc:0.701528+0.00283914
[4]	train-auc:0.729756+0.00199823	test-auc:0.704376+0.00314715
[5]	train-auc:0.732566+0.0011085	test-auc:0.706236+0.00445656
[6]	train-auc:0.734346+0.000920211	test-auc:0.707268+0.00484968
[7]	train-auc:0.735829+0.000514441	test-auc:0.708117+0.00505061
[8]	train-auc:0.736871+0.000395667	test-auc:0.70883+0.00436885
[9]	train-auc:0.738099+0.000306364	test-auc:0.709714+0.0052724
[10]	train-auc:0.740508+0.000866488	test-auc:0.710616+0.00517437
[11]	train-auc:0.741294+0.000721944	test-auc:0.711154+0.00475873
[12]	train-auc:0.742619+0.00108132	test-auc:0.711367+0.0053606
[13]	train-auc:0.743249+0.00114331	test-auc:0.712102+0.00543585
[14]	train-auc:0.743972+0.00147611	test-auc:0.71247+0.00577115
[15]	train-auc:0.745182+0.00173482	test-auc:0.71287

[128]	train-auc:0.829049+0.00227155	test-auc:0.73704+0.00151919
[129]	train-auc:0.829656+0.00213083	test-auc:0.737075+0.00162046
[130]	train-auc:0.829729+0.00222016	test-auc:0.737056+0.00164355
[131]	train-auc:0.830154+0.0020813	test-auc:0.737026+0.00162532
[132]	train-auc:0.830593+0.00229673	test-auc:0.737091+0.00161111
[133]	train-auc:0.830954+0.00234417	test-auc:0.737223+0.00163189
[134]	train-auc:0.831234+0.00231029	test-auc:0.737254+0.00160296
[135]	train-auc:0.831608+0.00234277	test-auc:0.737339+0.00167212
[136]	train-auc:0.831863+0.00237846	test-auc:0.737269+0.0016337
[137]	train-auc:0.832069+0.00239619	test-auc:0.737291+0.00161998
[138]	train-auc:0.832363+0.00254071	test-auc:0.737251+0.00160275
[139]	train-auc:0.832861+0.00203977	test-auc:0.737317+0.00165906
[140]	train-auc:0.833266+0.00214668	test-auc:0.737268+0.00175135
[141]	train-auc:0.833756+0.00209252	test-auc:0.737269+0.00180646
[142]	train-auc:0.834207+0.00205271	test-auc:0.737394+0.0017391
[143]	train-auc:0.834454+0.00

[255]	train-auc:0.862655+0.000493359	test-auc:0.739407+0.00238544
[256]	train-auc:0.86292+0.000415125	test-auc:0.73945+0.00238609
[257]	train-auc:0.863177+0.000629634	test-auc:0.739431+0.00240881
[258]	train-auc:0.863256+0.000612998	test-auc:0.739431+0.00242435
[259]	train-auc:0.863605+0.000550098	test-auc:0.739428+0.00229889
[260]	train-auc:0.863716+0.000678472	test-auc:0.739422+0.00233209
[261]	train-auc:0.863884+0.000755638	test-auc:0.739432+0.00237783
[262]	train-auc:0.864043+0.000810165	test-auc:0.739412+0.00233229
[263]	train-auc:0.86438+0.000941329	test-auc:0.739433+0.00236172
[264]	train-auc:0.864698+0.000890465	test-auc:0.73938+0.00231399
[265]	train-auc:0.864889+0.000956723	test-auc:0.739336+0.00230989


0.736215 = maxd_6, gamma_8

0.738909

In [101]:
srav = xgb.XGBClassifier(**params)
srav.fit(xtrain, ytrain)
pred_to_file(srav.predict_proba(xtest)[:, 1])

Валидация: 0.739336

Лидерборд: 0.73931072

### lightgbm

In [107]:
params = {
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_threads': 4,
    "metric": 'auc',
    'random_seed': 666,
}
n_rounds = 10000

lgb_train = lgb.Dataset(xtrain, label=ytrain, free_raw_data=False)

In [105]:
result = lgb.cv(params,
                lgb_train,
                n_rounds,
                folds=skf.split(xtrain, ytrain),
                early_stopping_rounds=10,
                verbose_eval=1)

[1]	cv_agg's auc: 0.686362 + 0.0102095
[2]	cv_agg's auc: 0.694582 + 0.0102184
[3]	cv_agg's auc: 0.698598 + 0.00900459
[4]	cv_agg's auc: 0.700617 + 0.00866554
[5]	cv_agg's auc: 0.703707 + 0.00742721
[6]	cv_agg's auc: 0.708083 + 0.00831141
[7]	cv_agg's auc: 0.709926 + 0.00937122
[8]	cv_agg's auc: 0.711865 + 0.00918754
[9]	cv_agg's auc: 0.713788 + 0.00988344
[10]	cv_agg's auc: 0.715976 + 0.00935391
[11]	cv_agg's auc: 0.71732 + 0.00861337
[12]	cv_agg's auc: 0.718068 + 0.00921695
[13]	cv_agg's auc: 0.719177 + 0.00923234
[14]	cv_agg's auc: 0.720281 + 0.00898898
[15]	cv_agg's auc: 0.721359 + 0.00895576
[16]	cv_agg's auc: 0.722496 + 0.00864592
[17]	cv_agg's auc: 0.724102 + 0.00897849
[18]	cv_agg's auc: 0.724801 + 0.00895238
[19]	cv_agg's auc: 0.726207 + 0.00868798
[20]	cv_agg's auc: 0.727395 + 0.00849264
[21]	cv_agg's auc: 0.728462 + 0.00842135
[22]	cv_agg's auc: 0.729353 + 0.00894822
[23]	cv_agg's auc: 0.730253 + 0.00889804
[24]	cv_agg's auc: 0.730941 + 0.00840724
[25]	cv_agg's auc: 0.731411 

In [106]:
def check_train_score(params, lgb_data, data, target, kf, num_rounds):
    roc_auc = []
    for train, val in kf.split(data,target):
        temp_lgb_train = lgb_data.subset(train)
        temp_lgb_val = lgb_data.subset(val)
        temp_model = lgb.train(params, temp_lgb_train, num_rounds, verbose_eval=num_rounds)
        roc_auc.append([roc_auc_score(target[train],temp_model.predict(data.loc[train])), roc_auc_score(target[val],temp_model.predict(data.loc[val]))])
    return np.mean(roc_auc, axis=0)

In [108]:
check_train_score(params, lgb_train, xtrain, ytrain, skf, 63)

array([0.84364971, 0.74128117])

In [174]:
params = {
    #default
    'objective': 'binary',
    'learning_rate': 0.05,
    'num_threads': -1,
    "metric": 'auc',
    'random_seed': 666,

    #regularization
#     'l2-leaf-reg': 6,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'subsample_freq': 1
}

result = lgb.cv(params, lgb_train, n_rounds, folds=skf.split(xtrain, ytrain),
                early_stopping_rounds=10, verbose_eval=1)

[1]	cv_agg's auc: 0.673938 + 0.011513
[2]	cv_agg's auc: 0.688957 + 0.0122199
[3]	cv_agg's auc: 0.696471 + 0.0114683
[4]	cv_agg's auc: 0.699559 + 0.010747
[5]	cv_agg's auc: 0.704846 + 0.00984765
[6]	cv_agg's auc: 0.707711 + 0.00987932
[7]	cv_agg's auc: 0.709628 + 0.00889804
[8]	cv_agg's auc: 0.711373 + 0.00909009
[9]	cv_agg's auc: 0.71227 + 0.00869173
[10]	cv_agg's auc: 0.713157 + 0.00797793
[11]	cv_agg's auc: 0.714049 + 0.00821536
[12]	cv_agg's auc: 0.714752 + 0.00877605
[13]	cv_agg's auc: 0.715498 + 0.00803839
[14]	cv_agg's auc: 0.71641 + 0.00812718
[15]	cv_agg's auc: 0.71707 + 0.0083202
[16]	cv_agg's auc: 0.717688 + 0.00832247
[17]	cv_agg's auc: 0.718024 + 0.00877073
[18]	cv_agg's auc: 0.71873 + 0.00856991
[19]	cv_agg's auc: 0.719576 + 0.00872096
[20]	cv_agg's auc: 0.720089 + 0.00869216
[21]	cv_agg's auc: 0.720568 + 0.00851186
[22]	cv_agg's auc: 0.721616 + 0.0085683
[23]	cv_agg's auc: 0.72246 + 0.00848291
[24]	cv_agg's auc: 0.723118 + 0.00841485
[25]	cv_agg's auc: 0.723361 + 0.008551

In [172]:
score = check_train_score(params, lgb_train, xtrain, ytrain, skf, len(result['auc-mean']))
print("Score = ", score)

Score =  [0.87054079 0.74361692]


In [56]:
allDone()

[0.87654715, 0.74290873] - basic

[0.87054079 0.74361692] - reg 0.6

0.74361692 = gamma 7

In [175]:
srav = lgb.LGBMClassifier(**params)
srav.fit(xtrain, ytrain)
pred_to_file(srav.predict_proba(xtest)[:, 1])

Валидация: 0.74361692

Лидерборд: 0.75059557

### catboost

In [20]:
cat_feats = np.where(train.dtypes == 'object')[0].tolist()

In [21]:
cat_feats

[]

In [53]:
params = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 10000,
    'learning_rate': 0.03,
    'random_seed': 666,
    'use_best_model': False,
    'od_wait': 30,
    'od_type': 'Iter',
    'thread_count': 4
}

ctb_data = ctb.Pool(xtrain, ytrain)
ctb.cv(ctb_data, params, stratified=True, fold_count=10,  seed=42, logging_level='Verbose')

0:	learn: 0.6370177	test: 0.6313921	best: 0.6313921 (0)	total: 1.17s	remaining: 3h 14m 56s
1:	learn: 0.6536677	test: 0.6469203	best: 0.6469203 (1)	total: 2.21s	remaining: 3h 4m 5s
2:	learn: 0.6590051	test: 0.6530747	best: 0.6530747 (2)	total: 3.29s	remaining: 3h 2m 49s
3:	learn: 0.6600420	test: 0.6546512	best: 0.6546512 (3)	total: 4.52s	remaining: 3h 8m 19s
4:	learn: 0.6643871	test: 0.6581843	best: 0.6581843 (4)	total: 5.71s	remaining: 3h 10m 14s
5:	learn: 0.6669361	test: 0.6598830	best: 0.6598830 (5)	total: 6.78s	remaining: 3h 8m 17s
6:	learn: 0.6686493	test: 0.6611007	best: 0.6611007 (6)	total: 7.89s	remaining: 3h 7m 48s
7:	learn: 0.6696587	test: 0.6617686	best: 0.6617686 (7)	total: 8.99s	remaining: 3h 7m 6s
8:	learn: 0.6707438	test: 0.6630825	best: 0.6630825 (8)	total: 10.1s	remaining: 3h 6m 22s
9:	learn: 0.6743348	test: 0.6669248	best: 0.6669248 (9)	total: 11.2s	remaining: 3h 6m 29s
10:	learn: 0.6757185	test: 0.6687350	best: 0.6687350 (10)	total: 12.3s	remaining: 3h 5m 41s
11:	lear

89:	learn: 0.7289655	test: 0.7154395	best: 0.7154395 (89)	total: 2m 23s	remaining: 4h 22m 30s
90:	learn: 0.7294340	test: 0.7158967	best: 0.7158967 (90)	total: 2m 24s	remaining: 4h 22m 3s
91:	learn: 0.7297532	test: 0.7161642	best: 0.7161642 (91)	total: 2m 25s	remaining: 4h 21m 30s
92:	learn: 0.7302736	test: 0.7167246	best: 0.7167246 (92)	total: 2m 26s	remaining: 4h 20m 46s
93:	learn: 0.7305534	test: 0.7170524	best: 0.7170524 (93)	total: 2m 28s	remaining: 4h 20m 43s
94:	learn: 0.7309508	test: 0.7173961	best: 0.7173961 (94)	total: 2m 29s	remaining: 4h 20m 17s
95:	learn: 0.7312062	test: 0.7176361	best: 0.7176361 (95)	total: 2m 31s	remaining: 4h 19m 40s
96:	learn: 0.7315239	test: 0.7178195	best: 0.7178195 (96)	total: 2m 32s	remaining: 4h 19m 21s
97:	learn: 0.7318672	test: 0.7181019	best: 0.7181019 (97)	total: 2m 33s	remaining: 4h 18m 41s
98:	learn: 0.7321485	test: 0.7183358	best: 0.7183358 (98)	total: 2m 34s	remaining: 4h 18m 12s
99:	learn: 0.7324261	test: 0.7186690	best: 0.7186690 (99)	tot

176:	learn: 0.7494054	test: 0.7284047	best: 0.7284047 (176)	total: 4m 16s	remaining: 3h 56m 51s
177:	learn: 0.7496320	test: 0.7284951	best: 0.7284951 (177)	total: 4m 17s	remaining: 3h 56m 42s
178:	learn: 0.7498418	test: 0.7286620	best: 0.7286620 (178)	total: 4m 18s	remaining: 3h 56m 35s
179:	learn: 0.7499892	test: 0.7286898	best: 0.7286898 (179)	total: 4m 20s	remaining: 3h 56m 28s
180:	learn: 0.7501637	test: 0.7287271	best: 0.7287271 (180)	total: 4m 21s	remaining: 3h 56m 18s
181:	learn: 0.7503464	test: 0.7287884	best: 0.7287884 (181)	total: 4m 22s	remaining: 3h 56m 27s
182:	learn: 0.7505004	test: 0.7289323	best: 0.7289323 (182)	total: 4m 24s	remaining: 3h 56m 16s
183:	learn: 0.7506557	test: 0.7290500	best: 0.7290500 (183)	total: 4m 25s	remaining: 3h 56m 8s
184:	learn: 0.7508443	test: 0.7291194	best: 0.7291194 (184)	total: 4m 26s	remaining: 3h 55m 59s
185:	learn: 0.7510445	test: 0.7292386	best: 0.7292386 (185)	total: 4m 28s	remaining: 3h 55m 53s
186:	learn: 0.7512017	test: 0.7293180	bes

262:	learn: 0.7630950	test: 0.7335770	best: 0.7335770 (262)	total: 6m 10s	remaining: 3h 48m 22s
263:	learn: 0.7632322	test: 0.7336164	best: 0.7336164 (263)	total: 6m 11s	remaining: 3h 48m 15s
264:	learn: 0.7633673	test: 0.7336551	best: 0.7336551 (264)	total: 6m 12s	remaining: 3h 48m 9s
265:	learn: 0.7635035	test: 0.7336555	best: 0.7336555 (265)	total: 6m 13s	remaining: 3h 48m 5s
266:	learn: 0.7636431	test: 0.7336562	best: 0.7336562 (266)	total: 6m 15s	remaining: 3h 47m 59s
267:	learn: 0.7637909	test: 0.7337103	best: 0.7337103 (267)	total: 6m 16s	remaining: 3h 47m 56s
268:	learn: 0.7639838	test: 0.7337903	best: 0.7337903 (268)	total: 6m 18s	remaining: 3h 48m 24s
269:	learn: 0.7641461	test: 0.7338529	best: 0.7338529 (269)	total: 6m 21s	remaining: 3h 48m 59s
270:	learn: 0.7642535	test: 0.7338926	best: 0.7338926 (270)	total: 6m 22s	remaining: 3h 48m 51s
271:	learn: 0.7643990	test: 0.7339303	best: 0.7339303 (271)	total: 6m 23s	remaining: 3h 48m 48s
272:	learn: 0.7645657	test: 0.7339943	best

348:	learn: 0.7754460	test: 0.7372437	best: 0.7372437 (348)	total: 8m 13s	remaining: 3h 47m 15s
349:	learn: 0.7756026	test: 0.7372853	best: 0.7372853 (349)	total: 8m 14s	remaining: 3h 47m 10s
350:	learn: 0.7757748	test: 0.7373315	best: 0.7373315 (350)	total: 8m 15s	remaining: 3h 47m 11s
351:	learn: 0.7759165	test: 0.7373565	best: 0.7373565 (351)	total: 8m 17s	remaining: 3h 47m 3s
352:	learn: 0.7760575	test: 0.7373709	best: 0.7373709 (352)	total: 8m 18s	remaining: 3h 47m 1s
353:	learn: 0.7762072	test: 0.7374321	best: 0.7374321 (353)	total: 8m 19s	remaining: 3h 46m 55s
354:	learn: 0.7763846	test: 0.7374373	best: 0.7374373 (354)	total: 8m 20s	remaining: 3h 46m 50s
355:	learn: 0.7765321	test: 0.7374680	best: 0.7374680 (355)	total: 8m 22s	remaining: 3h 46m 59s
356:	learn: 0.7766644	test: 0.7374478	best: 0.7374680 (355)	total: 8m 24s	remaining: 3h 47m 7s
357:	learn: 0.7768159	test: 0.7374754	best: 0.7374754 (357)	total: 8m 26s	remaining: 3h 47m 12s
358:	learn: 0.7769570	test: 0.7375559	best:

...

774:	learn: 0.8270584	test: 0.7435086	best: 0.7435574 (766)	total: 17m 37s	remaining: 3h 29m 43s
775:	learn: 0.8271493	test: 0.7435126	best: 0.7435574 (766)	total: 17m 38s	remaining: 3h 29m 39s
776:	learn: 0.8272511	test: 0.7435392	best: 0.7435574 (766)	total: 17m 39s	remaining: 3h 29m 37s
777:	learn: 0.8273345	test: 0.7435782	best: 0.7435782 (777)	total: 17m 40s	remaining: 3h 29m 32s
778:	learn: 0.8274325	test: 0.7435784	best: 0.7435784 (778)	total: 17m 41s	remaining: 3h 29m 28s
779:	learn: 0.8275350	test: 0.7435712	best: 0.7435784 (778)	total: 17m 43s	remaining: 3h 29m 25s
780:	learn: 0.8276250	test: 0.7435786	best: 0.7435786 (780)	total: 17m 44s	remaining: 3h 29m 20s
781:	learn: 0.8277191	test: 0.7435682	best: 0.7435786 (780)	total: 17m 45s	remaining: 3h 29m 15s
782:	learn: 0.8278204	test: 0.7435604	best: 0.7435786 (780)	total: 17m 46s	remaining: 3h 29m 12s
783:	learn: 0.8279102	test: 0.7435326	best: 0.7435786 (780)	total: 17m 47s	remaining: 3h 29m 8s
784:	learn: 0.8279937	test: 0.7

859:	learn: 0.8348615	test: 0.7438247	best: 0.7438513 (853)	total: 19m 25s	remaining: 3h 26m 27s
860:	learn: 0.8349334	test: 0.7438552	best: 0.7438552 (860)	total: 19m 26s	remaining: 3h 26m 24s
861:	learn: 0.8350379	test: 0.7438833	best: 0.7438833 (861)	total: 19m 27s	remaining: 3h 26m 21s
862:	learn: 0.8351023	test: 0.7439060	best: 0.7439060 (862)	total: 19m 29s	remaining: 3h 26m 18s
863:	learn: 0.8351927	test: 0.7439214	best: 0.7439214 (863)	total: 19m 30s	remaining: 3h 26m 14s
864:	learn: 0.8352875	test: 0.7439051	best: 0.7439214 (863)	total: 19m 31s	remaining: 3h 26m 11s
865:	learn: 0.8353678	test: 0.7439061	best: 0.7439214 (863)	total: 19m 32s	remaining: 3h 26m 8s
866:	learn: 0.8354494	test: 0.7439069	best: 0.7439214 (863)	total: 19m 33s	remaining: 3h 26m 4s
867:	learn: 0.8355290	test: 0.7438922	best: 0.7439214 (863)	total: 19m 34s	remaining: 3h 26m 1s
868:	learn: 0.8356111	test: 0.7439018	best: 0.7439214 (863)	total: 19m 36s	remaining: 3h 25m 57s
869:	learn: 0.8356926	test: 0.743

944:	learn: 0.8419062	test: 0.7443744	best: 0.7444054 (943)	total: 21m 12s	remaining: 3h 23m 10s
945:	learn: 0.8419752	test: 0.7443570	best: 0.7444054 (943)	total: 21m 13s	remaining: 3h 23m 8s
946:	learn: 0.8420642	test: 0.7443770	best: 0.7444054 (943)	total: 21m 14s	remaining: 3h 23m 6s
947:	learn: 0.8421597	test: 0.7443823	best: 0.7444054 (943)	total: 21m 16s	remaining: 3h 23m 4s
948:	learn: 0.8422408	test: 0.7444107	best: 0.7444107 (948)	total: 21m 17s	remaining: 3h 23m 3s
949:	learn: 0.8423293	test: 0.7444408	best: 0.7444408 (949)	total: 21m 19s	remaining: 3h 23m 5s
950:	learn: 0.8424106	test: 0.7444477	best: 0.7444477 (950)	total: 21m 20s	remaining: 3h 23m 7s
951:	learn: 0.8425117	test: 0.7443952	best: 0.7444477 (950)	total: 21m 22s	remaining: 3h 23m 10s
952:	learn: 0.8425801	test: 0.7443872	best: 0.7444477 (950)	total: 21m 24s	remaining: 3h 23m 10s
953:	learn: 0.8426754	test: 0.7443919	best: 0.7444477 (950)	total: 21m 25s	remaining: 3h 23m 8s
954:	learn: 0.8427326	test: 0.7443681

1029:	learn: 0.8485137	test: 0.7446862	best: 0.7446862 (1029)	total: 23m 3s	remaining: 3h 20m 51s
1030:	learn: 0.8485886	test: 0.7447164	best: 0.7447164 (1030)	total: 23m 5s	remaining: 3h 20m 50s
1031:	learn: 0.8486681	test: 0.7447355	best: 0.7447355 (1031)	total: 23m 6s	remaining: 3h 20m 46s
1032:	learn: 0.8487361	test: 0.7447290	best: 0.7447355 (1031)	total: 23m 7s	remaining: 3h 20m 44s
1033:	learn: 0.8488073	test: 0.7447220	best: 0.7447355 (1031)	total: 23m 8s	remaining: 3h 20m 41s
1034:	learn: 0.8488847	test: 0.7447315	best: 0.7447355 (1031)	total: 23m 9s	remaining: 3h 20m 39s
1035:	learn: 0.8489581	test: 0.7447118	best: 0.7447355 (1031)	total: 23m 11s	remaining: 3h 20m 36s
1036:	learn: 0.8490155	test: 0.7447212	best: 0.7447355 (1031)	total: 23m 12s	remaining: 3h 20m 33s
1037:	learn: 0.8490724	test: 0.7447371	best: 0.7447371 (1037)	total: 23m 13s	remaining: 3h 20m 31s
1038:	learn: 0.8491537	test: 0.7447548	best: 0.7447548 (1038)	total: 23m 14s	remaining: 3h 20m 29s
1039:	learn: 0.8

,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-AUC-mean,train-AUC-std,train-Logloss-mean,train-Logloss-std
0,0.631392,0.011036,0.676869,0.000152,0.637018,0.005043,0.676810,0.000134
1,0.646920,0.008507,0.662064,0.000180,0.653668,0.006938,0.661906,0.000273
2,0.653075,0.007741,0.648007,0.000246,0.659005,0.004113,0.647833,0.000281
3,0.654651,0.008771,0.634909,0.000311,0.660042,0.003378,0.634720,0.000286
4,0.658184,0.009333,0.622804,0.000360,0.664387,0.004066,0.622565,0.000381
5,0.659883,0.008570,0.611410,0.000373,0.666936,0.003303,0.611113,0.000398
6,0.661101,0.009684,0.600928,0.000372,0.668649,0.002622,0.600560,0.000387
7,0.661769,0.009896,0.591196,0.000395,0.669659,0.002293,0.590785,0.000337
8,0.663082,0.009232,0.581675,0.000411,0.670744,0.002356,0.581230,0.000339
9,0.666925,0.008741,0.572602,0.000432,0.674335,0.002261,0.572135,0.000345


In [54]:
allDone()

In [37]:
ctb.CatBoostClassifier?

In [55]:
params = {
    #default
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 10000,
    'learning_rate': 0.03,
    'random_seed': 666,
    'use_best_model': False,
    'od_wait': 50,
    'od_type': 'Iter',
    'thread_count': 4,

    #regularization
    'random_strength': 0.1,
    'rsm': 0.5,
    'l2_leaf_reg': 5,
    'depth': 7
}

ctb_data = ctb.Pool(xtrain, ytrain)
ctb.cv(ctb_data, params, stratified=True, fold_count=10, seed=666, logging_level='Verbose')

0:	learn: 0.6744275	test: 0.6668883	best: 0.6668883 (0)	total: 805ms	remaining: 2h 14m 11s
1:	learn: 0.6982632	test: 0.6911815	best: 0.6911815 (1)	total: 1.84s	remaining: 2h 33m 1s
2:	learn: 0.7039450	test: 0.6978850	best: 0.6978850 (2)	total: 2.6s	remaining: 2h 24m 21s
3:	learn: 0.7053086	test: 0.6978627	best: 0.6978850 (2)	total: 3.33s	remaining: 2h 18m 54s
4:	learn: 0.7077003	test: 0.6988097	best: 0.6988097 (4)	total: 4.15s	remaining: 2h 18m 9s
5:	learn: 0.7077823	test: 0.6983546	best: 0.6988097 (4)	total: 5.11s	remaining: 2h 21m 45s
6:	learn: 0.7088923	test: 0.6989478	best: 0.6989478 (6)	total: 6.15s	remaining: 2h 26m 20s
7:	learn: 0.7100917	test: 0.7003443	best: 0.7003443 (7)	total: 7.16s	remaining: 2h 28m 57s
8:	learn: 0.7099876	test: 0.7001731	best: 0.7003443 (7)	total: 8.04s	remaining: 2h 28m 51s
9:	learn: 0.7102858	test: 0.7009648	best: 0.7009648 (9)	total: 8.76s	remaining: 2h 25m 50s
10:	learn: 0.7101093	test: 0.7006998	best: 0.7009648 (9)	total: 9.46s	remaining: 2h 23m 14s
1

89:	learn: 0.7510689	test: 0.7283722	best: 0.7283722 (89)	total: 1m 23s	remaining: 2h 32m 47s
90:	learn: 0.7514670	test: 0.7285911	best: 0.7285911 (90)	total: 1m 24s	remaining: 2h 32m 44s
91:	learn: 0.7518026	test: 0.7287225	best: 0.7287225 (91)	total: 1m 25s	remaining: 2h 32m 56s
92:	learn: 0.7520703	test: 0.7288661	best: 0.7288661 (92)	total: 1m 26s	remaining: 2h 32m 54s
93:	learn: 0.7523920	test: 0.7290158	best: 0.7290158 (93)	total: 1m 27s	remaining: 2h 33m
94:	learn: 0.7527031	test: 0.7291624	best: 0.7291624 (94)	total: 1m 28s	remaining: 2h 33m 14s
95:	learn: 0.7530793	test: 0.7293368	best: 0.7293368 (95)	total: 1m 28s	remaining: 2h 33m
96:	learn: 0.7533658	test: 0.7293972	best: 0.7293972 (96)	total: 1m 29s	remaining: 2h 32m 43s
97:	learn: 0.7536645	test: 0.7295437	best: 0.7295437 (97)	total: 1m 30s	remaining: 2h 32m 25s
98:	learn: 0.7539731	test: 0.7296631	best: 0.7296631 (98)	total: 1m 31s	remaining: 2h 32m 23s
99:	learn: 0.7542554	test: 0.7298427	best: 0.7298427 (99)	total: 1m 

176:	learn: 0.7736116	test: 0.7350828	best: 0.7350828 (176)	total: 2m 42s	remaining: 2h 30m 25s
177:	learn: 0.7738439	test: 0.7351260	best: 0.7351260 (177)	total: 2m 43s	remaining: 2h 30m 23s
178:	learn: 0.7740579	test: 0.7351742	best: 0.7351742 (178)	total: 2m 44s	remaining: 2h 30m 17s
179:	learn: 0.7742561	test: 0.7352719	best: 0.7352719 (179)	total: 2m 45s	remaining: 2h 30m 14s
180:	learn: 0.7744677	test: 0.7353579	best: 0.7353579 (180)	total: 2m 46s	remaining: 2h 30m 5s
181:	learn: 0.7746769	test: 0.7354155	best: 0.7354155 (181)	total: 2m 46s	remaining: 2h 29m 58s
182:	learn: 0.7749264	test: 0.7354441	best: 0.7354441 (182)	total: 2m 47s	remaining: 2h 29m 48s
183:	learn: 0.7751640	test: 0.7354508	best: 0.7354508 (183)	total: 2m 48s	remaining: 2h 29m 44s
184:	learn: 0.7753302	test: 0.7355218	best: 0.7355218 (184)	total: 2m 49s	remaining: 2h 29m 34s
185:	learn: 0.7755942	test: 0.7356408	best: 0.7356408 (185)	total: 2m 49s	remaining: 2h 29m 27s
186:	learn: 0.7758241	test: 0.7356720	bes

262:	learn: 0.7914106	test: 0.7391658	best: 0.7391658 (262)	total: 3m 55s	remaining: 2h 25m 13s
263:	learn: 0.7915709	test: 0.7392667	best: 0.7392667 (263)	total: 3m 56s	remaining: 2h 25m 12s
264:	learn: 0.7917777	test: 0.7393098	best: 0.7393098 (264)	total: 3m 56s	remaining: 2h 25m 4s
265:	learn: 0.7919832	test: 0.7393402	best: 0.7393402 (265)	total: 3m 57s	remaining: 2h 24m 59s
266:	learn: 0.7921278	test: 0.7393310	best: 0.7393402 (265)	total: 3m 58s	remaining: 2h 24m 55s
267:	learn: 0.7923149	test: 0.7394011	best: 0.7394011 (267)	total: 3m 59s	remaining: 2h 24m 55s
268:	learn: 0.7925111	test: 0.7394540	best: 0.7394540 (268)	total: 4m	remaining: 2h 24m 51s
269:	learn: 0.7926970	test: 0.7394826	best: 0.7394826 (269)	total: 4m	remaining: 2h 24m 43s
270:	learn: 0.7929212	test: 0.7395311	best: 0.7395311 (270)	total: 4m 1s	remaining: 2h 24m 38s
271:	learn: 0.7930753	test: 0.7395421	best: 0.7395421 (271)	total: 4m 2s	remaining: 2h 24m 36s
272:	learn: 0.7932817	test: 0.7395706	best: 0.73957

348:	learn: 0.8070916	test: 0.7410668	best: 0.7410668 (348)	total: 5m 10s	remaining: 2h 22m 55s
349:	learn: 0.8072638	test: 0.7411274	best: 0.7411274 (349)	total: 5m 10s	remaining: 2h 22m 52s
350:	learn: 0.8074171	test: 0.7411408	best: 0.7411408 (350)	total: 5m 11s	remaining: 2h 22m 48s
351:	learn: 0.8075439	test: 0.7411514	best: 0.7411514 (351)	total: 5m 12s	remaining: 2h 22m 44s
352:	learn: 0.8077065	test: 0.7411644	best: 0.7411644 (352)	total: 5m 13s	remaining: 2h 22m 43s
353:	learn: 0.8078834	test: 0.7411960	best: 0.7411960 (353)	total: 5m 14s	remaining: 2h 22m 39s
354:	learn: 0.8080915	test: 0.7411601	best: 0.7411960 (353)	total: 5m 15s	remaining: 2h 22m 38s
355:	learn: 0.8083000	test: 0.7411836	best: 0.7411960 (353)	total: 5m 15s	remaining: 2h 22m 32s
356:	learn: 0.8084762	test: 0.7412058	best: 0.7412058 (356)	total: 5m 16s	remaining: 2h 22m 31s
357:	learn: 0.8086864	test: 0.7412448	best: 0.7412448 (357)	total: 5m 17s	remaining: 2h 22m 26s
358:	learn: 0.8088304	test: 0.7412417	be

...

606:	learn: 0.8442146	test: 0.7444004	best: 0.7444004 (606)	total: 8m 36s	remaining: 2h 13m 9s
607:	learn: 0.8443669	test: 0.7443666	best: 0.7444004 (606)	total: 8m 36s	remaining: 2h 13m 6s
608:	learn: 0.8444779	test: 0.7443913	best: 0.7444004 (606)	total: 8m 37s	remaining: 2h 13m 4s
609:	learn: 0.8446063	test: 0.7444053	best: 0.7444053 (609)	total: 8m 38s	remaining: 2h 13m 5s
610:	learn: 0.8447381	test: 0.7444026	best: 0.7444053 (609)	total: 8m 39s	remaining: 2h 13m 3s
611:	learn: 0.8448897	test: 0.7444536	best: 0.7444536 (611)	total: 8m 40s	remaining: 2h 12m 59s
612:	learn: 0.8450033	test: 0.7444265	best: 0.7444536 (611)	total: 8m 40s	remaining: 2h 12m 57s
613:	learn: 0.8451176	test: 0.7444003	best: 0.7444536 (611)	total: 8m 41s	remaining: 2h 12m 57s
614:	learn: 0.8452386	test: 0.7444000	best: 0.7444536 (611)	total: 8m 42s	remaining: 2h 12m 55s
615:	learn: 0.8453487	test: 0.7444309	best: 0.7444536 (611)	total: 8m 43s	remaining: 2h 12m 53s
616:	learn: 0.8454560	test: 0.7444048	best: 0

692:	learn: 0.8541027	test: 0.7447562	best: 0.7448595 (685)	total: 9m 48s	remaining: 2h 11m 47s
693:	learn: 0.8542241	test: 0.7447470	best: 0.7448595 (685)	total: 9m 49s	remaining: 2h 11m 49s
694:	learn: 0.8543239	test: 0.7447535	best: 0.7448595 (685)	total: 9m 50s	remaining: 2h 11m 49s
695:	learn: 0.8544242	test: 0.7447260	best: 0.7448595 (685)	total: 9m 51s	remaining: 2h 11m 49s
696:	learn: 0.8545103	test: 0.7447290	best: 0.7448595 (685)	total: 9m 52s	remaining: 2h 11m 50s
697:	learn: 0.8545997	test: 0.7447143	best: 0.7448595 (685)	total: 9m 53s	remaining: 2h 11m 49s
698:	learn: 0.8546963	test: 0.7447248	best: 0.7448595 (685)	total: 9m 54s	remaining: 2h 11m 48s
699:	learn: 0.8547992	test: 0.7447833	best: 0.7448595 (685)	total: 9m 55s	remaining: 2h 11m 46s
700:	learn: 0.8548888	test: 0.7447769	best: 0.7448595 (685)	total: 9m 56s	remaining: 2h 11m 47s
701:	learn: 0.8549888	test: 0.7447819	best: 0.7448595 (685)	total: 9m 57s	remaining: 2h 11m 48s
702:	learn: 0.8550744	test: 0.7447838	be

777:	learn: 0.8630075	test: 0.7450592	best: 0.7450805 (769)	total: 11m 1s	remaining: 2h 10m 35s
778:	learn: 0.8630942	test: 0.7450428	best: 0.7450805 (769)	total: 11m 1s	remaining: 2h 10m 33s
779:	learn: 0.8632001	test: 0.7450691	best: 0.7450805 (769)	total: 11m 2s	remaining: 2h 10m 31s
780:	learn: 0.8632989	test: 0.7450899	best: 0.7450899 (780)	total: 11m 3s	remaining: 2h 10m 31s
781:	learn: 0.8634304	test: 0.7451237	best: 0.7451237 (781)	total: 11m 4s	remaining: 2h 10m 29s
782:	learn: 0.8635363	test: 0.7451518	best: 0.7451518 (782)	total: 11m 5s	remaining: 2h 10m 28s
783:	learn: 0.8636396	test: 0.7451694	best: 0.7451694 (783)	total: 11m 5s	remaining: 2h 10m 26s
784:	learn: 0.8637120	test: 0.7451648	best: 0.7451694 (783)	total: 11m 6s	remaining: 2h 10m 25s
785:	learn: 0.8637981	test: 0.7451587	best: 0.7451694 (783)	total: 11m 7s	remaining: 2h 10m 24s
786:	learn: 0.8638963	test: 0.7451570	best: 0.7451694 (783)	total: 11m 8s	remaining: 2h 10m 21s
787:	learn: 0.8640169	test: 0.7451801	be

863:	learn: 0.8709230	test: 0.7453688	best: 0.7454538 (828)	total: 12m 9s	remaining: 2h 8m 31s
864:	learn: 0.8709760	test: 0.7453715	best: 0.7454538 (828)	total: 12m 10s	remaining: 2h 8m 32s
865:	learn: 0.8710639	test: 0.7453821	best: 0.7454538 (828)	total: 12m 11s	remaining: 2h 8m 31s
866:	learn: 0.8711636	test: 0.7453790	best: 0.7454538 (828)	total: 12m 11s	remaining: 2h 8m 30s
867:	learn: 0.8712563	test: 0.7453707	best: 0.7454538 (828)	total: 12m 12s	remaining: 2h 8m 29s
868:	learn: 0.8713601	test: 0.7453561	best: 0.7454538 (828)	total: 12m 13s	remaining: 2h 8m 29s
869:	learn: 0.8714433	test: 0.7453564	best: 0.7454538 (828)	total: 12m 14s	remaining: 2h 8m 28s
870:	learn: 0.8715365	test: 0.7453171	best: 0.7454538 (828)	total: 12m 15s	remaining: 2h 8m 27s
871:	learn: 0.8716436	test: 0.7453505	best: 0.7454538 (828)	total: 12m 16s	remaining: 2h 8m 25s
872:	learn: 0.8717179	test: 0.7453230	best: 0.7454538 (828)	total: 12m 17s	remaining: 2h 8m 25s
873:	learn: 0.8717908	test: 0.7453216	bes

,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-AUC-mean,train-AUC-std,train-Logloss-mean,train-Logloss-std
0,0.666888,0.011841,0.675670,0.000186,0.674428,0.004116,0.675543,0.000127
1,0.691182,0.015429,0.658944,0.000399,0.698263,0.004284,0.658713,0.000125
2,0.697885,0.015154,0.643502,0.000447,0.703945,0.003367,0.643229,0.000270
3,0.697863,0.014089,0.629200,0.000567,0.705309,0.002166,0.628821,0.000276
4,0.698810,0.014640,0.615671,0.000766,0.707700,0.002415,0.615152,0.000280
5,0.698355,0.014448,0.603224,0.000816,0.707782,0.002136,0.602577,0.000333
6,0.698948,0.015484,0.591611,0.000963,0.708892,0.001451,0.590827,0.000371
7,0.700344,0.015468,0.580680,0.001028,0.710092,0.001831,0.579797,0.000474
8,0.700173,0.015412,0.570563,0.001036,0.709988,0.001546,0.569557,0.000548
9,0.700965,0.015224,0.561062,0.001076,0.710286,0.001731,0.559988,0.000607


reg(0.4, 5) = 0.740256

reg(0.4, 5, 5) = 0.740538

reg(0.4, 5, 4) = 0.739602

reg(0.5, 4, 4) = 0.739714


reg(0.4, 5, 5, lr=0.05) = 0.739038

0.741376

0.741166

In [8]:
params = {
    #default
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 585,
    'learning_rate': 0.03,
    'random_seed': 666,
    'use_best_model': False,
    'od_wait': 50,
    'od_type': 'Iter',
    'thread_count': 4,

    #regularization
    'random_strength': 0.1,
    'rsm': 0.5,
    'l2_leaf_reg': 5,
    'depth': 7
}

srav = ctb.CatBoostClassifier(**params)
srav.fit(xtrain, ytrain)
pred_to_file(srav.predict_proba(xtest)[:, 1])

0:	learn: 0.6875360	total: 133ms	remaining: 1m 17s
1:	learn: 0.7018576	total: 207ms	remaining: 1m
2:	learn: 0.7008242	total: 286ms	remaining: 55.4s
3:	learn: 0.7002193	total: 398ms	remaining: 57.9s
4:	learn: 0.7013198	total: 490ms	remaining: 56.8s
5:	learn: 0.7042903	total: 565ms	remaining: 54.6s
6:	learn: 0.7046057	total: 663ms	remaining: 54.7s
7:	learn: 0.7057208	total: 741ms	remaining: 53.5s
8:	learn: 0.7067813	total: 815ms	remaining: 52.1s
9:	learn: 0.7076754	total: 920ms	remaining: 52.9s
10:	learn: 0.7092624	total: 1000ms	remaining: 52.2s
11:	learn: 0.7105169	total: 1.11s	remaining: 53.1s
12:	learn: 0.7110161	total: 1.24s	remaining: 54.6s
13:	learn: 0.7113615	total: 1.37s	remaining: 55.8s
14:	learn: 0.7128461	total: 1.44s	remaining: 54.9s
15:	learn: 0.7140001	total: 1.54s	remaining: 54.9s
16:	learn: 0.7137539	total: 1.64s	remaining: 54.7s
17:	learn: 0.7147524	total: 1.75s	remaining: 55.2s
18:	learn: 0.7151452	total: 1.85s	remaining: 55.2s
19:	learn: 0.7161395	total: 1.94s	remainin

163:	learn: 0.7673811	total: 14.9s	remaining: 38.2s
164:	learn: 0.7676353	total: 15s	remaining: 38.1s
165:	learn: 0.7678244	total: 15s	remaining: 38s
166:	learn: 0.7680998	total: 15.1s	remaining: 37.9s
167:	learn: 0.7682854	total: 15.2s	remaining: 37.8s
168:	learn: 0.7685489	total: 15.3s	remaining: 37.7s
169:	learn: 0.7688294	total: 15.4s	remaining: 37.6s
170:	learn: 0.7689617	total: 15.5s	remaining: 37.5s
171:	learn: 0.7691322	total: 15.6s	remaining: 37.4s
172:	learn: 0.7693421	total: 15.6s	remaining: 37.2s
173:	learn: 0.7695063	total: 15.7s	remaining: 37.1s
174:	learn: 0.7696697	total: 15.8s	remaining: 37s
175:	learn: 0.7698718	total: 15.9s	remaining: 36.9s
176:	learn: 0.7700842	total: 16s	remaining: 36.8s
177:	learn: 0.7703719	total: 16s	remaining: 36.7s
178:	learn: 0.7705187	total: 16.1s	remaining: 36.6s
179:	learn: 0.7707286	total: 16.2s	remaining: 36.5s
180:	learn: 0.7709575	total: 16.3s	remaining: 36.4s
181:	learn: 0.7712616	total: 16.5s	remaining: 36.4s
182:	learn: 0.7715529	to

322:	learn: 0.7970749	total: 29.9s	remaining: 24.2s
323:	learn: 0.7972609	total: 30s	remaining: 24.1s
324:	learn: 0.7974367	total: 30.1s	remaining: 24.1s
325:	learn: 0.7976195	total: 30.2s	remaining: 24s
326:	learn: 0.7977205	total: 30.2s	remaining: 23.9s
327:	learn: 0.7978925	total: 30.3s	remaining: 23.8s
328:	learn: 0.7979890	total: 30.4s	remaining: 23.7s
329:	learn: 0.7982126	total: 30.5s	remaining: 23.6s
330:	learn: 0.7984711	total: 30.6s	remaining: 23.5s
331:	learn: 0.7986049	total: 30.7s	remaining: 23.4s
332:	learn: 0.7989152	total: 30.8s	remaining: 23.3s
333:	learn: 0.7990681	total: 30.9s	remaining: 23.2s
334:	learn: 0.7992517	total: 31s	remaining: 23.1s
335:	learn: 0.7994526	total: 31s	remaining: 23s
336:	learn: 0.7996405	total: 31.1s	remaining: 22.9s
337:	learn: 0.7998202	total: 31.2s	remaining: 22.8s
338:	learn: 0.7999890	total: 31.3s	remaining: 22.7s
339:	learn: 0.8001934	total: 31.4s	remaining: 22.6s
340:	learn: 0.8002840	total: 31.5s	remaining: 22.5s
341:	learn: 0.8004860	

483:	learn: 0.8214821	total: 45s	remaining: 9.39s
484:	learn: 0.8215529	total: 45.1s	remaining: 9.3s
485:	learn: 0.8216886	total: 45.2s	remaining: 9.21s
486:	learn: 0.8218178	total: 45.3s	remaining: 9.11s
487:	learn: 0.8219309	total: 45.4s	remaining: 9.02s
488:	learn: 0.8220182	total: 45.5s	remaining: 8.92s
489:	learn: 0.8221602	total: 45.5s	remaining: 8.83s
490:	learn: 0.8222652	total: 45.6s	remaining: 8.73s
491:	learn: 0.8224215	total: 45.7s	remaining: 8.63s
492:	learn: 0.8225047	total: 45.8s	remaining: 8.54s
493:	learn: 0.8225734	total: 45.9s	remaining: 8.45s
494:	learn: 0.8227284	total: 45.9s	remaining: 8.35s
495:	learn: 0.8228659	total: 46s	remaining: 8.26s
496:	learn: 0.8230216	total: 46.1s	remaining: 8.17s
497:	learn: 0.8230706	total: 46.2s	remaining: 8.07s
498:	learn: 0.8232732	total: 46.3s	remaining: 7.98s
499:	learn: 0.8234288	total: 46.4s	remaining: 7.88s
500:	learn: 0.8235678	total: 46.5s	remaining: 7.79s
501:	learn: 0.8237723	total: 46.6s	remaining: 7.7s
502:	learn: 0.8239

Валидация: 0.741376	

Лидерборд: 0.75110026

### ansmble

In [8]:
params_xgb = {
    'objective': 'reg:logistic',
    'eta': 0.05,
    'silent': 1,
    "nthread": -1,
    "random_seed": 666,
    "eval_metric": 'auc',
    'gamma': 8,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'tree_method': 'hist'
}
params_lgb = {
    'objective': 'binary',
    'learning_rate': 0.05,
    'num_threads': -1,
    "metric": 'auc',
    'random_seed': 666,
    'colsample_bytree': 0.6,
    'subsample': 0.6,
    'subsample_freq': 1
}
params_cat = {
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 585,
    'learning_rate': 0.03,
    'random_seed': 666,
    'use_best_model': False,
    'od_wait': 50,
    'od_type': 'Iter',
    'thread_count': 4,
    'random_strength': 0.1,
    'rsm': 0.5,
    'l2_leaf_reg': 5,
    'depth': 7
}

xgb_m = xgb.XGBClassifier(**params_xgb)
# xgb_m.fit(xtrain, ytrain)
lgb_m = lgb.LGBMClassifier(**params_lgb)
# lgb_m.fit(xtrain, ytrain)
cat_m = ctb.CatBoostClassifier(**params_cat)
# cat_m.fit(xtrain, ytrain)

In [9]:
xgb_preds = []
lgb_preds = []
cat_preds = []

for train, val in skf.split(xtrain, ytrain):
    xgb_m.fit(xtrain.iloc[train], ytrain[train])
    lgb_m.fit(xtrain.iloc[train], ytrain[train])
    cat_m.fit(xtrain.iloc[train], ytrain[train])
    xgb_preds.append([ytrain[val], xgb_m.predict_proba(xtrain.iloc[val])[:,1]])
    lgb_preds.append([ytrain[val], lgb_m.predict_proba(xtrain.iloc[val])[:,1]])
    cat_preds.append([ytrain[val], cat_m.predict_proba(xtrain.iloc[val])[:,1]])

0:	learn: 0.6895242	total: 125ms	remaining: 1m 12s
1:	learn: 0.7094881	total: 196ms	remaining: 57.3s
2:	learn: 0.7116993	total: 286ms	remaining: 55.4s
3:	learn: 0.7111532	total: 396ms	remaining: 57.5s
4:	learn: 0.7133096	total: 502ms	remaining: 58.2s
5:	learn: 0.7130652	total: 575ms	remaining: 55.5s
6:	learn: 0.7144768	total: 701ms	remaining: 57.9s
7:	learn: 0.7152909	total: 783ms	remaining: 56.4s
8:	learn: 0.7150792	total: 896ms	remaining: 57.3s
9:	learn: 0.7159011	total: 1.05s	remaining: 1m
10:	learn: 0.7171717	total: 1.15s	remaining: 1m
11:	learn: 0.7166254	total: 1.23s	remaining: 58.8s
12:	learn: 0.7179270	total: 1.33s	remaining: 58.8s
13:	learn: 0.7175288	total: 1.41s	remaining: 57.7s
14:	learn: 0.7177660	total: 1.48s	remaining: 56.3s
15:	learn: 0.7176346	total: 1.55s	remaining: 55.2s
16:	learn: 0.7179267	total: 1.65s	remaining: 55.1s
17:	learn: 0.7181389	total: 1.72s	remaining: 54.1s
18:	learn: 0.7184068	total: 1.83s	remaining: 54.5s
19:	learn: 0.7187122	total: 1.9s	remaining: 53

163:	learn: 0.7709164	total: 15.5s	remaining: 39.8s
164:	learn: 0.7711833	total: 15.6s	remaining: 39.7s
165:	learn: 0.7713687	total: 15.7s	remaining: 39.6s
166:	learn: 0.7716276	total: 15.8s	remaining: 39.5s
167:	learn: 0.7718442	total: 15.8s	remaining: 39.3s
168:	learn: 0.7719515	total: 15.9s	remaining: 39.2s
169:	learn: 0.7721851	total: 16s	remaining: 39.1s
170:	learn: 0.7723887	total: 16.1s	remaining: 38.9s
171:	learn: 0.7725879	total: 16.2s	remaining: 38.8s
172:	learn: 0.7728886	total: 16.3s	remaining: 38.8s
173:	learn: 0.7732456	total: 16.4s	remaining: 38.8s
174:	learn: 0.7734244	total: 16.5s	remaining: 38.8s
175:	learn: 0.7738511	total: 16.6s	remaining: 38.6s
176:	learn: 0.7740335	total: 16.7s	remaining: 38.5s
177:	learn: 0.7741832	total: 16.8s	remaining: 38.4s
178:	learn: 0.7744362	total: 16.9s	remaining: 38.2s
179:	learn: 0.7746155	total: 16.9s	remaining: 38.1s
180:	learn: 0.7748266	total: 17s	remaining: 38s
181:	learn: 0.7750366	total: 17.1s	remaining: 37.8s
182:	learn: 0.7752

324:	learn: 0.8037949	total: 29.5s	remaining: 23.6s
325:	learn: 0.8040861	total: 29.5s	remaining: 23.5s
326:	learn: 0.8042279	total: 29.6s	remaining: 23.4s
327:	learn: 0.8043846	total: 29.7s	remaining: 23.3s
328:	learn: 0.8044874	total: 29.8s	remaining: 23.2s
329:	learn: 0.8046183	total: 29.9s	remaining: 23.1s
330:	learn: 0.8047152	total: 30s	remaining: 23s
331:	learn: 0.8048387	total: 30s	remaining: 22.9s
332:	learn: 0.8049944	total: 30.1s	remaining: 22.8s
333:	learn: 0.8051009	total: 30.2s	remaining: 22.7s
334:	learn: 0.8053637	total: 30.3s	remaining: 22.6s
335:	learn: 0.8055812	total: 30.4s	remaining: 22.5s
336:	learn: 0.8057703	total: 30.4s	remaining: 22.4s
337:	learn: 0.8059213	total: 30.5s	remaining: 22.3s
338:	learn: 0.8059861	total: 30.6s	remaining: 22.2s
339:	learn: 0.8062862	total: 30.7s	remaining: 22.1s
340:	learn: 0.8063502	total: 30.8s	remaining: 22s
341:	learn: 0.8065354	total: 30.8s	remaining: 21.9s
342:	learn: 0.8066721	total: 30.9s	remaining: 21.8s
343:	learn: 0.806899

484:	learn: 0.8283171	total: 42.9s	remaining: 8.84s
485:	learn: 0.8285457	total: 42.9s	remaining: 8.75s
486:	learn: 0.8286732	total: 43s	remaining: 8.66s
487:	learn: 0.8288109	total: 43.1s	remaining: 8.57s
488:	learn: 0.8289168	total: 43.2s	remaining: 8.48s
489:	learn: 0.8290977	total: 43.3s	remaining: 8.39s
490:	learn: 0.8291936	total: 43.4s	remaining: 8.3s
491:	learn: 0.8293481	total: 43.4s	remaining: 8.21s
492:	learn: 0.8294582	total: 43.5s	remaining: 8.12s
493:	learn: 0.8295879	total: 43.6s	remaining: 8.03s
494:	learn: 0.8296908	total: 43.7s	remaining: 7.94s
495:	learn: 0.8298134	total: 43.8s	remaining: 7.86s
496:	learn: 0.8299917	total: 43.9s	remaining: 7.78s
497:	learn: 0.8301263	total: 44.1s	remaining: 7.7s
498:	learn: 0.8301991	total: 44.1s	remaining: 7.61s
499:	learn: 0.8303825	total: 44.2s	remaining: 7.52s
500:	learn: 0.8305626	total: 44.3s	remaining: 7.43s
501:	learn: 0.8306811	total: 44.4s	remaining: 7.34s
502:	learn: 0.8307561	total: 44.5s	remaining: 7.25s
503:	learn: 0.83

59:	learn: 0.7396813	total: 5.04s	remaining: 44.1s
60:	learn: 0.7398627	total: 5.12s	remaining: 44s
61:	learn: 0.7402672	total: 5.18s	remaining: 43.7s
62:	learn: 0.7407176	total: 5.26s	remaining: 43.6s
63:	learn: 0.7413315	total: 5.34s	remaining: 43.5s
64:	learn: 0.7417789	total: 5.42s	remaining: 43.4s
65:	learn: 0.7422578	total: 5.5s	remaining: 43.3s
66:	learn: 0.7425382	total: 5.58s	remaining: 43.2s
67:	learn: 0.7428089	total: 5.65s	remaining: 42.9s
68:	learn: 0.7432040	total: 5.74s	remaining: 43s
69:	learn: 0.7434908	total: 5.82s	remaining: 42.8s
70:	learn: 0.7438818	total: 5.9s	remaining: 42.7s
71:	learn: 0.7443679	total: 5.98s	remaining: 42.6s
72:	learn: 0.7445745	total: 6.07s	remaining: 42.6s
73:	learn: 0.7449910	total: 6.14s	remaining: 42.4s
74:	learn: 0.7453439	total: 6.23s	remaining: 42.3s
75:	learn: 0.7456326	total: 6.3s	remaining: 42.2s
76:	learn: 0.7458561	total: 6.37s	remaining: 42s
77:	learn: 0.7461964	total: 6.46s	remaining: 42s
78:	learn: 0.7468760	total: 6.53s	remainin

...

455:	learn: 0.8241542	total: 39.6s	remaining: 11.2s
456:	learn: 0.8242804	total: 39.7s	remaining: 11.1s
457:	learn: 0.8245097	total: 39.8s	remaining: 11s
458:	learn: 0.8246795	total: 39.9s	remaining: 10.9s
459:	learn: 0.8249104	total: 39.9s	remaining: 10.9s
460:	learn: 0.8250910	total: 40s	remaining: 10.8s
461:	learn: 0.8252869	total: 40.1s	remaining: 10.7s
462:	learn: 0.8253427	total: 40.3s	remaining: 10.6s
463:	learn: 0.8255560	total: 40.4s	remaining: 10.5s
464:	learn: 0.8256660	total: 40.5s	remaining: 10.5s
465:	learn: 0.8257410	total: 40.7s	remaining: 10.4s
466:	learn: 0.8258857	total: 40.8s	remaining: 10.3s
467:	learn: 0.8260668	total: 40.8s	remaining: 10.2s
468:	learn: 0.8262984	total: 40.9s	remaining: 10.1s
469:	learn: 0.8264475	total: 41s	remaining: 10s
470:	learn: 0.8266215	total: 41.1s	remaining: 9.94s
471:	learn: 0.8268721	total: 41.1s	remaining: 9.85s
472:	learn: 0.8270053	total: 41.2s	remaining: 9.76s
473:	learn: 0.8270996	total: 41.3s	remaining: 9.67s
474:	learn: 0.827227

32:	learn: 0.7269694	total: 2.51s	remaining: 42s
33:	learn: 0.7270789	total: 2.58s	remaining: 41.9s
34:	learn: 0.7275732	total: 2.66s	remaining: 41.8s
35:	learn: 0.7281208	total: 2.73s	remaining: 41.6s
36:	learn: 0.7285717	total: 2.8s	remaining: 41.5s
37:	learn: 0.7291072	total: 2.88s	remaining: 41.5s
38:	learn: 0.7292611	total: 2.95s	remaining: 41.3s
39:	learn: 0.7299818	total: 3.03s	remaining: 41.3s
40:	learn: 0.7310033	total: 3.11s	remaining: 41.2s
41:	learn: 0.7311081	total: 3.19s	remaining: 41.3s
42:	learn: 0.7316652	total: 3.28s	remaining: 41.3s
43:	learn: 0.7322417	total: 3.34s	remaining: 41.1s
44:	learn: 0.7323709	total: 3.42s	remaining: 41.1s
45:	learn: 0.7327478	total: 3.49s	remaining: 40.9s
46:	learn: 0.7335249	total: 3.56s	remaining: 40.8s
47:	learn: 0.7335977	total: 3.64s	remaining: 40.7s
48:	learn: 0.7342298	total: 3.71s	remaining: 40.6s
49:	learn: 0.7343180	total: 3.78s	remaining: 40.4s
50:	learn: 0.7345314	total: 3.86s	remaining: 40.4s
51:	learn: 0.7353018	total: 3.93s	

194:	learn: 0.7757758	total: 15.5s	remaining: 31s
195:	learn: 0.7760530	total: 15.6s	remaining: 31s
196:	learn: 0.7761942	total: 15.7s	remaining: 30.9s
197:	learn: 0.7764026	total: 15.8s	remaining: 30.8s
198:	learn: 0.7766411	total: 15.8s	remaining: 30.7s
199:	learn: 0.7768185	total: 15.9s	remaining: 30.6s
200:	learn: 0.7770694	total: 16s	remaining: 30.5s
201:	learn: 0.7772478	total: 16.1s	remaining: 30.4s
202:	learn: 0.7773728	total: 16.1s	remaining: 30.4s
203:	learn: 0.7774847	total: 16.3s	remaining: 30.4s
204:	learn: 0.7775874	total: 16.4s	remaining: 30.3s
205:	learn: 0.7777623	total: 16.5s	remaining: 30.4s
206:	learn: 0.7779351	total: 16.6s	remaining: 30.3s
207:	learn: 0.7782416	total: 16.7s	remaining: 30.2s
208:	learn: 0.7783327	total: 16.7s	remaining: 30.1s
209:	learn: 0.7786298	total: 16.8s	remaining: 30s
210:	learn: 0.7789728	total: 16.9s	remaining: 29.9s
211:	learn: 0.7792555	total: 17s	remaining: 29.9s
212:	learn: 0.7795603	total: 17s	remaining: 29.8s
213:	learn: 0.7797263	to

353:	learn: 0.8067923	total: 28.3s	remaining: 18.5s
354:	learn: 0.8069207	total: 28.4s	remaining: 18.4s
355:	learn: 0.8071301	total: 28.6s	remaining: 18.4s
356:	learn: 0.8072977	total: 28.7s	remaining: 18.3s
357:	learn: 0.8073928	total: 28.8s	remaining: 18.2s
358:	learn: 0.8075226	total: 28.9s	remaining: 18.2s
359:	learn: 0.8076259	total: 28.9s	remaining: 18.1s
360:	learn: 0.8077886	total: 29s	remaining: 18s
361:	learn: 0.8078920	total: 29.1s	remaining: 17.9s
362:	learn: 0.8080709	total: 29.1s	remaining: 17.8s
363:	learn: 0.8082313	total: 29.2s	remaining: 17.7s
364:	learn: 0.8085217	total: 29.3s	remaining: 17.7s
365:	learn: 0.8086149	total: 29.4s	remaining: 17.6s
366:	learn: 0.8087712	total: 29.4s	remaining: 17.5s
367:	learn: 0.8088995	total: 29.5s	remaining: 17.4s
368:	learn: 0.8090409	total: 29.6s	remaining: 17.3s
369:	learn: 0.8091780	total: 29.7s	remaining: 17.2s
370:	learn: 0.8094188	total: 29.7s	remaining: 17.2s
371:	learn: 0.8095996	total: 29.8s	remaining: 17.1s
372:	learn: 0.80

513:	learn: 0.8314253	total: 41.3s	remaining: 5.7s
514:	learn: 0.8315042	total: 41.3s	remaining: 5.62s
515:	learn: 0.8317098	total: 41.4s	remaining: 5.54s
516:	learn: 0.8319116	total: 41.5s	remaining: 5.46s
517:	learn: 0.8319955	total: 41.6s	remaining: 5.38s
518:	learn: 0.8321120	total: 41.7s	remaining: 5.3s
519:	learn: 0.8322734	total: 41.7s	remaining: 5.22s
520:	learn: 0.8323418	total: 41.8s	remaining: 5.14s
521:	learn: 0.8324814	total: 41.9s	remaining: 5.06s
522:	learn: 0.8326433	total: 42s	remaining: 4.98s
523:	learn: 0.8327515	total: 42.1s	remaining: 4.89s
524:	learn: 0.8328537	total: 42.1s	remaining: 4.81s
525:	learn: 0.8329720	total: 42.2s	remaining: 4.74s
526:	learn: 0.8331486	total: 42.3s	remaining: 4.65s
527:	learn: 0.8333401	total: 42.4s	remaining: 4.57s
528:	learn: 0.8334767	total: 42.4s	remaining: 4.49s
529:	learn: 0.8336572	total: 42.5s	remaining: 4.41s
530:	learn: 0.8337052	total: 42.6s	remaining: 4.33s
531:	learn: 0.8338152	total: 42.7s	remaining: 4.25s
532:	learn: 0.83

In [10]:
np.mean([roc_auc_score(xgb_p[0], np.cbrt(xgb_p[1]*lgb_p[1]*cat_p[1])) for xgb_p, lgb_p, cat_p in
         zip(xgb_preds, lgb_preds, cat_preds)])

0.7440321460148411

In [11]:
from scipy.stats import rankdata
np.mean([roc_auc_score(xgb_p[0], rankdata(xgb_p[1]) + rankdata(lgb_p[1]) + rankdata(cat_p[1]))
         for xgb_p, lgb_p, cat_p in
         zip(xgb_preds, lgb_preds, cat_preds)])

0.7436645525421046

In [46]:
mx = 0
best_i = 0
best_j = 0
best_k = 0
for i in np.arange(0.01, 0.99, 0.01):
    for j in np.arange(0.01, 1 - i - 0.01, 0.01):
        k = 1 - i - j
        score = np.mean([roc_auc_score(xgb_p[0], xgb_p[1]*i + lgb_p[1]*j + cat_p[1]*k) for xgb_p, lgb_p, cat_p in
                 zip(xgb_preds, lgb_preds, cat_preds)])
        if score > mx:
            mx = score
            best_i = i
            best_j = j
            best_k = k

In [47]:
mx, best_i, best_j, best_k

(0.746058951455258, 0.01, 0.28, 0.71)

In [12]:
mx = 0
best_i = 0
for i in np.arange(0.01, 1.0, 0.01):
    score = np.mean([roc_auc_score(lgb_p[0], lgb_p[1]*i + cat_p[1]*(1 - i)) for lgb_p, cat_p in
             zip(lgb_preds, cat_preds)])
    if score > mx:
        mx = score
        best_i = i

In [13]:
mx, best_i

(0.7460822744027666, 0.24000000000000002)

In [51]:
lgb_m.fit(xtrain, ytrain)
cat_m.fit(xtrain, ytrain)

0:	learn: 0.6875360	total: 72.1ms	remaining: 42.1s
1:	learn: 0.7018576	total: 147ms	remaining: 42.9s
2:	learn: 0.7008242	total: 224ms	remaining: 43.5s
3:	learn: 0.7002193	total: 318ms	remaining: 46.1s
4:	learn: 0.7013198	total: 420ms	remaining: 48.7s
5:	learn: 0.7042903	total: 497ms	remaining: 48s
6:	learn: 0.7046057	total: 603ms	remaining: 49.8s
7:	learn: 0.7057208	total: 692ms	remaining: 49.9s
8:	learn: 0.7067813	total: 770ms	remaining: 49.3s
9:	learn: 0.7076754	total: 875ms	remaining: 50.3s
10:	learn: 0.7092624	total: 950ms	remaining: 49.6s
11:	learn: 0.7105169	total: 1.03s	remaining: 49.3s
12:	learn: 0.7110161	total: 1.13s	remaining: 49.8s
13:	learn: 0.7113615	total: 1.23s	remaining: 50.2s
14:	learn: 0.7128461	total: 1.3s	remaining: 49.6s
15:	learn: 0.7140001	total: 1.43s	remaining: 50.8s
16:	learn: 0.7137539	total: 1.52s	remaining: 50.9s
17:	learn: 0.7147524	total: 1.6s	remaining: 50.4s
18:	learn: 0.7151452	total: 1.71s	remaining: 51s
19:	learn: 0.7161395	total: 1.79s	remaining: 5

161:	learn: 0.7670895	total: 16.3s	remaining: 42.6s
162:	learn: 0.7672121	total: 16.4s	remaining: 42.4s
163:	learn: 0.7673811	total: 16.5s	remaining: 42.2s
164:	learn: 0.7676353	total: 16.6s	remaining: 42.2s
165:	learn: 0.7678244	total: 16.6s	remaining: 42s
166:	learn: 0.7680998	total: 16.7s	remaining: 41.9s
167:	learn: 0.7682854	total: 16.9s	remaining: 41.8s
168:	learn: 0.7685489	total: 16.9s	remaining: 41.7s
169:	learn: 0.7688294	total: 17.1s	remaining: 41.6s
170:	learn: 0.7689617	total: 17.2s	remaining: 41.5s
171:	learn: 0.7691322	total: 17.2s	remaining: 41.4s
172:	learn: 0.7693421	total: 17.4s	remaining: 41.4s
173:	learn: 0.7695063	total: 17.5s	remaining: 41.4s
174:	learn: 0.7696697	total: 17.7s	remaining: 41.4s
175:	learn: 0.7698718	total: 17.7s	remaining: 41.2s
176:	learn: 0.7700842	total: 17.8s	remaining: 41.1s
177:	learn: 0.7703719	total: 17.9s	remaining: 41s
178:	learn: 0.7705187	total: 18s	remaining: 40.9s
179:	learn: 0.7707286	total: 18.1s	remaining: 40.8s
180:	learn: 0.7709

320:	learn: 0.7968262	total: 31.6s	remaining: 26s
321:	learn: 0.7969157	total: 31.7s	remaining: 25.9s
322:	learn: 0.7970749	total: 31.8s	remaining: 25.8s
323:	learn: 0.7972609	total: 31.9s	remaining: 25.7s
324:	learn: 0.7974367	total: 31.9s	remaining: 25.5s
325:	learn: 0.7976195	total: 32s	remaining: 25.4s
326:	learn: 0.7977205	total: 32.1s	remaining: 25.3s
327:	learn: 0.7978925	total: 32.2s	remaining: 25.2s
328:	learn: 0.7979890	total: 32.3s	remaining: 25.1s
329:	learn: 0.7982126	total: 32.3s	remaining: 25s
330:	learn: 0.7984711	total: 32.4s	remaining: 24.9s
331:	learn: 0.7986049	total: 32.5s	remaining: 24.8s
332:	learn: 0.7989152	total: 32.6s	remaining: 24.7s
333:	learn: 0.7990681	total: 32.7s	remaining: 24.6s
334:	learn: 0.7992517	total: 32.9s	remaining: 24.5s
335:	learn: 0.7994526	total: 33s	remaining: 24.4s
336:	learn: 0.7996405	total: 33s	remaining: 24.3s
337:	learn: 0.7998202	total: 33.1s	remaining: 24.2s
338:	learn: 0.7999890	total: 33.2s	remaining: 24.1s
339:	learn: 0.8001934	

481:	learn: 0.8211433	total: 49.5s	remaining: 10.6s
482:	learn: 0.8213571	total: 49.6s	remaining: 10.5s
483:	learn: 0.8214821	total: 49.8s	remaining: 10.4s
484:	learn: 0.8215529	total: 49.9s	remaining: 10.3s
485:	learn: 0.8216886	total: 50s	remaining: 10.2s
486:	learn: 0.8218178	total: 50s	remaining: 10.1s
487:	learn: 0.8219309	total: 50.2s	remaining: 9.97s
488:	learn: 0.8220182	total: 50.2s	remaining: 9.86s
489:	learn: 0.8221602	total: 50.4s	remaining: 9.77s
490:	learn: 0.8222652	total: 50.5s	remaining: 9.66s
491:	learn: 0.8224215	total: 50.6s	remaining: 9.56s
492:	learn: 0.8225047	total: 50.7s	remaining: 9.46s
493:	learn: 0.8225734	total: 50.8s	remaining: 9.36s
494:	learn: 0.8227284	total: 50.9s	remaining: 9.25s
495:	learn: 0.8228659	total: 51s	remaining: 9.15s
496:	learn: 0.8230216	total: 51.1s	remaining: 9.06s
497:	learn: 0.8230706	total: 51.3s	remaining: 8.96s
498:	learn: 0.8232732	total: 51.4s	remaining: 8.86s
499:	learn: 0.8234288	total: 51.5s	remaining: 8.75s
500:	learn: 0.8235

In [52]:
pred_to_file(0.24*(lgb_m.predict_proba(xtest)[:, 1])
              + 0.76*(cat_m.predict_proba(xtest)[:, 1]))

Валидация: 0.7460822744027666

Лидерборд: 0.75203264

### stacking

In [14]:
xtrain["lgb_preds"] = 0
xtrain["cat_preds"] = 0
for i, fold in enumerate(skf.split(xtrain, ytrain)):
    tr, val = fold[0], fold[1]
    xtrain.iloc[val, -2] = lgb_preds[i][1]
    xtrain.iloc[val, -1] = cat_preds[i][1]

In [24]:
params_stack = {
    #default
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 10000,
    'learning_rate': 0.03,
    'random_seed': 666,
    'use_best_model': False,
    'od_wait': 50,
    'od_type': 'Iter',
    'thread_count': 4,

    #regularization
#     'random_strength': 0.1,
    'rsm': 0.2,
    'l2_leaf_reg': 7,
    'depth': 6
}

ctb_data = ctb.Pool(xtrain, ytrain)
ctb.cv(ctb_data, params_stack, stratified=True, seed=666, logging_level='Verbose')

0:	learn: 0.7041973	test: 0.7014161	best: 0.7014161 (0)	total: 116ms	remaining: 19m 22s
1:	learn: 0.7100095	test: 0.7069468	best: 0.7069468 (1)	total: 239ms	remaining: 19m 57s
2:	learn: 0.7119641	test: 0.7086448	best: 0.7086448 (2)	total: 349ms	remaining: 19m 21s
3:	learn: 0.7179898	test: 0.7118918	best: 0.7118918 (3)	total: 488ms	remaining: 20m 20s
4:	learn: 0.7207144	test: 0.7131579	best: 0.7131579 (4)	total: 620ms	remaining: 20m 38s
5:	learn: 0.7273353	test: 0.7184035	best: 0.7184035 (5)	total: 722ms	remaining: 20m 3s
6:	learn: 0.7237321	test: 0.7173316	best: 0.7184035 (5)	total: 813ms	remaining: 19m 19s
7:	learn: 0.7257191	test: 0.7190809	best: 0.7190809 (7)	total: 947ms	remaining: 19m 43s
8:	learn: 0.7271984	test: 0.7193488	best: 0.7193488 (8)	total: 1.05s	remaining: 19m 21s
9:	learn: 0.7339314	test: 0.7267209	best: 0.7267209 (9)	total: 1.14s	remaining: 19m 1s
10:	learn: 0.7355190	test: 0.7282987	best: 0.7282987 (10)	total: 1.28s	remaining: 19m 26s
11:	learn: 0.7338966	test: 0.726

93:	learn: 0.7533861	test: 0.7452049	best: 0.7452179 (76)	total: 10.4s	remaining: 18m 21s
94:	learn: 0.7534469	test: 0.7452151	best: 0.7452179 (76)	total: 10.5s	remaining: 18m 19s
95:	learn: 0.7534928	test: 0.7451928	best: 0.7452179 (76)	total: 10.7s	remaining: 18m 20s
96:	learn: 0.7535722	test: 0.7452017	best: 0.7452179 (76)	total: 10.8s	remaining: 18m 18s
97:	learn: 0.7536673	test: 0.7451591	best: 0.7452179 (76)	total: 10.9s	remaining: 18m 16s
98:	learn: 0.7537564	test: 0.7451613	best: 0.7452179 (76)	total: 10.9s	remaining: 18m 14s
99:	learn: 0.7538719	test: 0.7451888	best: 0.7452179 (76)	total: 11s	remaining: 18m 12s
100:	learn: 0.7538501	test: 0.7452431	best: 0.7452431 (100)	total: 11.1s	remaining: 18m 9s
101:	learn: 0.7539522	test: 0.7452676	best: 0.7452676 (101)	total: 11.3s	remaining: 18m 15s
102:	learn: 0.7539911	test: 0.7452804	best: 0.7452804 (102)	total: 11.5s	remaining: 18m 20s
103:	learn: 0.7540166	test: 0.7452798	best: 0.7452804 (102)	total: 11.6s	remaining: 18m 23s
104:	

183:	learn: 0.7608829	test: 0.7452686	best: 0.7454471 (140)	total: 20s	remaining: 17m 48s
184:	learn: 0.7609780	test: 0.7452884	best: 0.7454471 (140)	total: 20.1s	remaining: 17m 49s
185:	learn: 0.7610313	test: 0.7452809	best: 0.7454471 (140)	total: 20.3s	remaining: 17m 51s
186:	learn: 0.7611148	test: 0.7452925	best: 0.7454471 (140)	total: 20.7s	remaining: 18m 6s
187:	learn: 0.7612166	test: 0.7453048	best: 0.7454471 (140)	total: 20.8s	remaining: 18m 6s
188:	learn: 0.7613218	test: 0.7452574	best: 0.7454471 (140)	total: 20.9s	remaining: 18m 6s
189:	learn: 0.7614258	test: 0.7452372	best: 0.7454471 (140)	total: 21s	remaining: 18m 5s
190:	learn: 0.7615004	test: 0.7452599	best: 0.7454471 (140)	total: 21.2s	remaining: 18m 6s
Stopped by overfitting detector  (50 iterations wait)
191:	learn: 0.7615842	test: 0.7452563	best: 0.7454471 (140)	total: 21.3s	remaining: 18m 6s


,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-AUC-mean,train-AUC-std,train-Logloss-mean,train-Logloss-std
0,0.701416,0.006586,0.674017,0.000077,0.704197,0.001928,0.673994,0.000042
1,0.706947,0.005581,0.657186,0.000182,0.710010,0.002131,0.657132,0.000064
2,0.708645,0.005921,0.641641,0.000216,0.711964,0.002043,0.641575,0.000155
3,0.711892,0.002381,0.626661,0.000298,0.717990,0.002039,0.626494,0.000060
4,0.713158,0.004792,0.612879,0.000329,0.720714,0.001123,0.612695,0.000730
5,0.718403,0.007680,0.599633,0.000650,0.727335,0.006460,0.599332,0.000890
6,0.717332,0.004285,0.589498,0.000645,0.723732,0.007018,0.589184,0.000856
7,0.719081,0.004246,0.577993,0.000774,0.725719,0.007005,0.577617,0.000931
8,0.719349,0.003636,0.567631,0.000609,0.727198,0.005062,0.567190,0.000824
9,0.726721,0.007360,0.557239,0.000215,0.733931,0.002204,0.556746,0.000265


0.745180

0.745252

0.745256	

In [15]:
xtrain.head()

,1,2,3,4,6,7,8,10,12,13,...,338,339,340,341,342,343,344,345,lgb_preds,cat_preds
0,1,0,0,0,0,0.090909,0,1,1,0.461538,...,1,0,0,0.222222,1,1,1,1,0.461141,0.512637
1,1,0,0,1,0,0.090909,0,1,1,0.187500,...,1,0,0,0.111111,1,1,1,0,0.198834,0.200883
2,1,0,0,1,0,0.090909,0,1,1,0.200000,...,1,0,0,0.444444,1,1,1,1,0.270003,0.247557
3,1,0,0,1,0,0.136364,0,1,1,0.000000,...,0,1,0,0.222222,1,1,1,0,0.249285,0.258850
4,1,0,0,1,0,0.136364,0,1,1,0.650000,...,0,1,0,0.111111,1,1,1,1,0.586674,0.625407


In [16]:
lgb_m = lgb.LGBMClassifier(**params_lgb)
lgb_m.fit(xtrain.drop(['lgb_preds', 'cat_preds'], axis=1), ytrain)
cat_m = ctb.CatBoostClassifier(**params_cat)
cat_m.fit(xtrain.drop(['lgb_preds', 'cat_preds'], axis=1), ytrain)

0:	learn: 0.6875360	total: 68.7ms	remaining: 40.1s
1:	learn: 0.7018576	total: 146ms	remaining: 42.7s
2:	learn: 0.7008242	total: 217ms	remaining: 42.1s
3:	learn: 0.7002193	total: 298ms	remaining: 43.2s
4:	learn: 0.7013198	total: 399ms	remaining: 46.3s
5:	learn: 0.7042903	total: 482ms	remaining: 46.5s
6:	learn: 0.7046057	total: 598ms	remaining: 49.4s
7:	learn: 0.7057208	total: 679ms	remaining: 49s
8:	learn: 0.7067813	total: 754ms	remaining: 48.3s
9:	learn: 0.7076754	total: 867ms	remaining: 49.9s
10:	learn: 0.7092624	total: 948ms	remaining: 49.5s
11:	learn: 0.7105169	total: 1.04s	remaining: 49.6s
12:	learn: 0.7110161	total: 1.16s	remaining: 51s
13:	learn: 0.7113615	total: 1.25s	remaining: 51s
14:	learn: 0.7128461	total: 1.34s	remaining: 51s
15:	learn: 0.7140001	total: 1.45s	remaining: 51.5s
16:	learn: 0.7137539	total: 1.53s	remaining: 51s
17:	learn: 0.7147524	total: 1.6s	remaining: 50.5s
18:	learn: 0.7151452	total: 1.71s	remaining: 51s
19:	learn: 0.7161395	total: 1.79s	remaining: 50.6s
20

162:	learn: 0.7672121	total: 15.3s	remaining: 39.7s
163:	learn: 0.7673811	total: 15.4s	remaining: 39.6s
164:	learn: 0.7676353	total: 15.5s	remaining: 39.5s
165:	learn: 0.7678244	total: 15.6s	remaining: 39.3s
166:	learn: 0.7680998	total: 15.7s	remaining: 39.2s
167:	learn: 0.7682854	total: 15.8s	remaining: 39.1s
168:	learn: 0.7685489	total: 15.8s	remaining: 39s
169:	learn: 0.7688294	total: 15.9s	remaining: 38.8s
170:	learn: 0.7689617	total: 16s	remaining: 38.8s
171:	learn: 0.7691322	total: 16.1s	remaining: 38.7s
172:	learn: 0.7693421	total: 16.2s	remaining: 38.5s
173:	learn: 0.7695063	total: 16.3s	remaining: 38.4s
174:	learn: 0.7696697	total: 16.4s	remaining: 38.3s
175:	learn: 0.7698718	total: 16.4s	remaining: 38.2s
176:	learn: 0.7700842	total: 16.5s	remaining: 38.1s
177:	learn: 0.7703719	total: 16.6s	remaining: 38s
178:	learn: 0.7705187	total: 16.7s	remaining: 37.9s
179:	learn: 0.7707286	total: 16.8s	remaining: 37.8s
180:	learn: 0.7709575	total: 16.9s	remaining: 37.7s
181:	learn: 0.7712

321:	learn: 0.7969157	total: 31s	remaining: 25.3s
322:	learn: 0.7970749	total: 31.1s	remaining: 25.2s
323:	learn: 0.7972609	total: 31.2s	remaining: 25.1s
324:	learn: 0.7974367	total: 31.3s	remaining: 25s
325:	learn: 0.7976195	total: 31.4s	remaining: 24.9s
326:	learn: 0.7977205	total: 31.4s	remaining: 24.8s
327:	learn: 0.7978925	total: 31.5s	remaining: 24.7s
328:	learn: 0.7979890	total: 31.6s	remaining: 24.6s
329:	learn: 0.7982126	total: 31.7s	remaining: 24.5s
330:	learn: 0.7984711	total: 31.8s	remaining: 24.4s
331:	learn: 0.7986049	total: 31.9s	remaining: 24.3s
332:	learn: 0.7989152	total: 31.9s	remaining: 24.2s
333:	learn: 0.7990681	total: 32s	remaining: 24.1s
334:	learn: 0.7992517	total: 32.1s	remaining: 24s
335:	learn: 0.7994526	total: 32.2s	remaining: 23.9s
336:	learn: 0.7996405	total: 32.3s	remaining: 23.8s
337:	learn: 0.7998202	total: 32.4s	remaining: 23.7s
338:	learn: 0.7999890	total: 32.5s	remaining: 23.6s
339:	learn: 0.8001934	total: 32.6s	remaining: 23.5s
340:	learn: 0.800284

482:	learn: 0.8213571	total: 45.3s	remaining: 9.56s
483:	learn: 0.8214821	total: 45.3s	remaining: 9.46s
484:	learn: 0.8215529	total: 45.4s	remaining: 9.37s
485:	learn: 0.8216886	total: 45.6s	remaining: 9.28s
486:	learn: 0.8218178	total: 45.7s	remaining: 9.19s
487:	learn: 0.8219309	total: 45.8s	remaining: 9.1s
488:	learn: 0.8220182	total: 45.8s	remaining: 9s
489:	learn: 0.8221602	total: 45.9s	remaining: 8.9s
490:	learn: 0.8222652	total: 46s	remaining: 8.8s
491:	learn: 0.8224215	total: 46.1s	remaining: 8.71s
492:	learn: 0.8225047	total: 46.2s	remaining: 8.61s
493:	learn: 0.8225734	total: 46.2s	remaining: 8.52s
494:	learn: 0.8227284	total: 46.3s	remaining: 8.42s
495:	learn: 0.8228659	total: 46.4s	remaining: 8.32s
496:	learn: 0.8230216	total: 46.5s	remaining: 8.23s
497:	learn: 0.8230706	total: 46.5s	remaining: 8.13s
498:	learn: 0.8232732	total: 46.6s	remaining: 8.03s
499:	learn: 0.8234288	total: 46.7s	remaining: 7.94s
500:	learn: 0.8235678	total: 46.8s	remaining: 7.84s
501:	learn: 0.823772

In [17]:
xtest["lgb_preds"] = 0
xtest["cat_preds"] = 0
xtest['lgb_preds'] = lgb_m.predict_proba(xtest.drop(['lgb_preds', 'cat_preds'], axis=1))
xtest['cat_preds'] = cat_m.predict_proba(xtest.drop(['lgb_preds', 'cat_preds'], axis=1))

In [25]:
params_stack = {
    #default
    'loss_function':'Logloss',
    'eval_metric': 'AUC',
    'iterations': 192,
    'learning_rate': 0.03,
    'random_seed': 666,
    'use_best_model': False,
    'od_wait': 50,
    'od_type': 'Iter',
    'thread_count': 4,

    #regularization
#     'random_strength': 0.1,
    'rsm': 0.2,
    'l2_leaf_reg': 7,
    'depth': 6
}

In [29]:
xtest.head()

,1,2,3,4,6,7,8,10,12,13,...,338,339,340,341,342,343,344,345,lgb_preds,cat_preds
0,1,0,0,1,0,0.136364,0,1,1,0.555556,...,1,0,0,0.222222,1,1,1,1,0.845360,0.840906
1,1,0,0,1,0,0.181818,0,1,1,0.000000,...,1,0,0,0.000000,1,1,1,0,0.537744,0.461015
2,1,0,0,0,0,0.090909,0,1,1,0.000000,...,1,0,0,0.222222,1,1,1,0,0.715423,0.792724
3,1,0,0,1,0,0.090909,0,1,1,0.000000,...,1,0,0,0.000000,1,1,1,0,0.690829,0.659125
4,1,0,0,1,0,0.090909,0,1,1,1.000000,...,0,1,0,0.000000,1,1,1,1,0.369459,0.384455


In [26]:
cat_stack = ctb.CatBoostClassifier(**params_stack)
cat_stack.fit(xtrain, ytrain)

0:	learn: 0.6951423	total: 41.9ms	remaining: 8.01s
1:	learn: 0.7185392	total: 82.5ms	remaining: 7.83s
2:	learn: 0.7144718	total: 125ms	remaining: 7.84s
3:	learn: 0.7262545	total: 168ms	remaining: 7.9s
4:	learn: 0.7316266	total: 210ms	remaining: 7.84s
5:	learn: 0.7324125	total: 257ms	remaining: 7.98s
6:	learn: 0.7286799	total: 337ms	remaining: 8.9s
7:	learn: 0.7294538	total: 388ms	remaining: 8.92s
8:	learn: 0.7307947	total: 432ms	remaining: 8.78s
9:	learn: 0.7328167	total: 491ms	remaining: 8.94s
10:	learn: 0.7328909	total: 557ms	remaining: 9.17s
11:	learn: 0.7339658	total: 605ms	remaining: 9.07s
12:	learn: 0.7341520	total: 647ms	remaining: 8.9s
13:	learn: 0.7352744	total: 689ms	remaining: 8.76s
14:	learn: 0.7360722	total: 793ms	remaining: 9.36s
15:	learn: 0.7361729	total: 877ms	remaining: 9.64s
16:	learn: 0.7370366	total: 947ms	remaining: 9.74s
17:	learn: 0.7377339	total: 1.03s	remaining: 9.96s
18:	learn: 0.7376638	total: 1.13s	remaining: 10.3s
19:	learn: 0.7419285	total: 1.17s	remainin

164:	learn: 0.7545751	total: 9.13s	remaining: 1.49s
165:	learn: 0.7547499	total: 9.17s	remaining: 1.44s
166:	learn: 0.7548284	total: 9.21s	remaining: 1.38s
167:	learn: 0.7549139	total: 9.25s	remaining: 1.32s
168:	learn: 0.7549927	total: 9.3s	remaining: 1.27s
169:	learn: 0.7551116	total: 9.38s	remaining: 1.21s
170:	learn: 0.7551191	total: 9.44s	remaining: 1.16s
171:	learn: 0.7551746	total: 9.48s	remaining: 1.1s
172:	learn: 0.7552591	total: 9.54s	remaining: 1.05s
173:	learn: 0.7553504	total: 9.59s	remaining: 992ms
174:	learn: 0.7553994	total: 9.64s	remaining: 936ms
175:	learn: 0.7554494	total: 9.68s	remaining: 880ms
176:	learn: 0.7555219	total: 9.72s	remaining: 824ms
177:	learn: 0.7555363	total: 9.76s	remaining: 768ms
178:	learn: 0.7556407	total: 9.82s	remaining: 713ms
179:	learn: 0.7556934	total: 9.86s	remaining: 657ms
180:	learn: 0.7557248	total: 9.92s	remaining: 603ms
181:	learn: 0.7557694	total: 9.99s	remaining: 549ms
182:	learn: 0.7558718	total: 10.1s	remaining: 496ms
183:	learn: 0.

In [27]:
pred_to_file(cat_stack.predict_proba(xtest)[:, 1])